In [45]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline

from scikeras.wrappers import KerasClassifier   
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import pickle


In [46]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [47]:

data = data.drop(["RowNumber","CustomerId","Surname"],axis=1)

label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])


onehot = OneHotEncoder(handle_unknown='ignore')
geo_encoder = onehot.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoder, 
    columns=onehot.get_feature_names_out(['Geography'])
)


data = pd.concat([data.drop('Geography' ,axis=1) , geo_encoded_df] , axis=1)
data.head()

x = data.drop('Exited' , axis=1)
y = data['Exited']

x_train , x_test ,y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


#save the encoder
with open('label_encoder.pkl','wb') as file:
    pickle.dump(label_encoder,file)

with open('onehot.pkl','wb') as file:
    pickle.dump(onehot,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)



In [48]:
##Define a function to create the model and try different parameter(KerasClassifier)

def create_model(neurons=32 , layers=1):
    model = Sequential()
    model.add(Dense(neurons , activation='relu' ,input_shape = (x_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons , activation='relu')) ## hiddne layer
    
    model.add(Dense(1 , activation='sigmoid'))   ## o/p layer
    model.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy'])

    return model


In [49]:
## create a keras classifier -  used to build ANN  network

model = KerasClassifier(build_fn=create_model , neurons=32 , layers=1 , verbose=0)

In [52]:
param_grid = {
    'neurons' : [16,32,64,128],
    'layers'  : [1,2,3,4],
    'epochs'  : [50,100]

}

In [53]:
gridsearch = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_result = gridsearch.fit(x_train, y_train)

print(f"Best: {grid_result.best_score_:.4f} using {grid_result.best_params_}")


c:\Users\aryan\OneDrive\Desktop\ann classification\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\aryan\OneDrive\Desktop\ann classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best: 0.8373 using {'epochs': 50, 'layers': 1, 'neurons': 128}
